# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# pd.set_option('display.max_colwidth', -1)

In [ ]:
# list(set(df.columns) - set({'id', 'message', 'original', 'genre'}))

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('cleaned_data', engine)
X = df['message'].values
output_columns = list(set(df.columns) - set({'id', 'message', 'original', 'genre'}))
Y = df[output_columns].values

In [ ]:
Y

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """Take the following steps to clean and tokenize text data:
        1. Replace any urls in text with "urlplaceholder" to reduce the data noise caused by url
        2. Split text into tokens.
        3. lemmatize, normalize case, and strip leading and trailing white space.
    Args:
        text (str): the input text to clean and tokenize
    Returns:
        list of string: cleaned and tokenized list of words
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()   
    return [lemmatizer.lemmatize(token).lower().strip() for token in tokens]


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
%%time
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
# for columns in output_columns:
for i in range(Y_test.shape[1]):
    print(f"classification_report for category {output_columns[i]}:")
    print(classification_report(Y_test[:,i], Y_pred[:,i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
%%time
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__max_features': (None, 5000),
#     'clf__estimator__n_estimators': [50, 100],
#     'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3).fit(X_train, Y_train)

In [ ]:
cv.best_estimator_.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.